In [1]:
from functools import reduce

# Define Paulis and Hadamard
iden2 = Matrix.identity(2)
H = 1/sqrt(2)* Matrix([[1, 1],[1, -1]])
Z = Matrix([[1, 0], [0, -1]])
X = Matrix([[0, 1], [1, 0]])
Y = -I*Z*X

# Takes a variable number of operators and tensors them
tensor = lambda *projections: reduce(lambda a, b: a.tensor_product(b), projections)
# Trace of an operator
tr = lambda rho: rho.trace()
xor = lambda *args: reduce(lambda a, b: (a + b) % 2, args)
psi_gen = lambda theta, phi: Matrix([cos(theta), e^(I*phi)*sin(theta)]).transpose()

# Inputs in the GHZ game
inputs = [(0,0,0), (0, 1, 1), (1, 0, 1), (1, 1, 0)]

# Computational basis vectors 
z = Matrix([1, 0]).transpose() # |0>
o = Matrix([0, 1]).transpose() # |1>

# Hadamard basis vectors
p = 1/sqrt(2) * (z + o) # |+>
m = 1/sqrt(2) * (z - o) # |->

i_plus = 1/sqrt(2) * (z + I*o)
i_minus = 1/sqrt(2) * (z - I*o)

# Takes |psi> and returns |psi><psi|
proj = lambda psi: psi * psi.conjugate_transpose()
ketbra = lambda psi, phi: psi * phi.conjugate_transpose()

# Define projectors corresponding to computational and 
# Hadamard basis
zz = proj(z) # |0><0|
oo = proj(o) # |1><1|
pp = proj(p) # |+><+|
mm = proj(m) # |-><-|
i_pp = proj(i_plus)
i_mm = proj(i_minus)

# Projective measurement: 
# On input 0, measure in Hadamard basis
# On input 1, measure in Y basis
E = [[pp, mm], [i_pp, i_mm]]
#E2 = [[zz, oo], [pp, mm]]
# GHZ state
ghz_rho = proj(1/sqrt(2) * (tensor(z,z,z) + tensor(o,o,o)))

# Born rule for quantum probability
p = lambda y_A, y_B, y_C, x_A, x_B, x_C, E, F, G, rho: tr(tensor(E[x_A][y_A], F[x_B][y_B], G[x_C][y_C]) * rho)

# Probability of winning the GHZ game
pr_win = lambda PI1, PI2, PI3, rho: 1/4 * sum(
                                    p(y_A, y_B, y_C, x_A, x_B, x_C, PI1, PI2, PI3, rho) 
                                        for y_A in range(2) 
                                        for y_B in range(2) 
                                        for y_C in range(2)
                                        for x_A, x_B, x_C in inputs
                                        if xor(y_A, y_B, y_C) == (x_A | x_B | x_C)
                                  )
show(pr_win(E, E, E, ghz_rho))

1

In [3]:
theta, phi = var('theta phi')
q_ket = psi_gen(theta, phi)
Q = proj(q_ket)
E_jordan = [[pp, mm], [H*Q*H, iden2 - H*Q*H]]
show(solve(pr_win(E_jordan, E_jordan, E_jordan, ghz_rho).simplify_trig().differentiate(phi), phi))
show(solve(pr_win(E_jordan, E_jordan, E_jordan, ghz_rho).simplify_trig().subs(phi=pi/2).differentiate(theta), theta))
show(pr_win(E_jordan, E_jordan, E_jordan, ghz_rho).subs(phi=pi/2, theta=pi/4))

[phi == 1/2*pi, phi == pi, phi == -1/2*pi, phi == 0]

[theta == 0, theta == 3/4*pi, theta == 1/4*pi, theta == 1/2*pi]

1